In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
#loading channels sheet
from os import listdir
filepaths=["./channel/day=25/hour=17/"+f for f in listdir("./channel/day=25/hour=17/") if f.endswith('.csv')]
rawchan=[]
for filename in filepaths:
    temp=pd.read_csv(filename, header=0, sep="|")
    rawchan.append(temp)
df_channel=pd.concat(rawchan, axis=0, ignore_index=True)

In [ ]:
#loading video sheets
rawvideo=[]
filepaths=["./video/day=25/hour=17/"+f for f in listdir("./video/day=25/hour=17/") if f.endswith(".csv")]
for filename in filepaths:
    temp=pd.read_csv(filename, header=0, sep="|", error_bad_lines=False)
    rawvideo.append(temp)
df_video=pd.concat(rawvideo, axis=0, ignore_index=True)

In [4]:
#check variables
print(df_channel.columns)
print(df_video.columns)

Index(['id', 'title', 'collectedTime', 'category', 'sub_category',
       'description', 'publishedAt', 'thumbnailUrl', 'country', 'viewCount',
       'commentCount', 'subscriberCount', 'hiddenSubscriberCount',
       'videoCount', 'keywords', 'unsubscribedTrailer'],
      dtype='object')
Index(['id', 'collectedTime', 'publishedAt', 'channelId', 'title',
       'description', 'thumbnailUrl', 'tags', 'category', 'defaultLanguage',
       'duration', 'caption', 'viewCount', 'likeCount', 'dislikeCount',
       'commentCount'],
      dtype='object')


In [5]:
#change id@channel to channelId for ease of use
df_channel=df_channel.rename(columns={"id":"channelId"})

In [6]:
#date and time conversion
df_channel["collectedTime"]=pd.to_datetime(df_channel.collectedTime)
df_channel["publishedAt"]=pd.to_datetime(df_channel.publishedAt)
df_video["collectedTime"]=pd.to_datetime(df_video.collectedTime)
df_video["publishedAt"]=pd.to_datetime(df_video.publishedAt)

In [7]:
#sum total like and dislike from each videos
like=df_video.groupby(["channelId"]).sum()#get sum
like=like.drop(["viewCount", "commentCount"], axis=1).reset_index()#drop pointless columns and reset index

In [8]:
#merge calculated like/dislikes into chanel data set
df_channel=pd.merge(df_channel, like, how="outer")

In [9]:
#title and category @ channel
chtovid=df_channel[["channelId", "title", "category", "sub_category"]]#creating sub df
chtovid=chtovid.rename(columns={"channelId":"channelId", "title":"ch_title", "category":"ch_category", "sub_category":"ch_sub_category"})

In [10]:
#merge above title and category to video
df_video=pd.merge(df_video, chtovid, how="inner")

In [11]:
#parse date out of publishedAt from df_video
df_video["pubDate"]=df_video.publishedAt.astype(str).str[:-15]
df_video["pubDate"]=pd.to_datetime(df_video.pubDate)

In [12]:
#get recent 3-month data
df3M=df_video.set_index("pubDate").sort_index().last("3M")
df3M=df3M.reset_index()
#get recent 6-month data
df6M=df_video.set_index("pubDate").sort_index().last("6M")
df6M=df6M.reset_index()

In [13]:
#extract number of videos from the last three months using above df
chanvid=df3M.channelId.value_counts().rename_axis("channelId").reset_index(name="3M_video")
chanvid=chanvid.fillna(value=0)
chanvid6=df6M.channelId.value_counts().rename_axis("channelId").reset_index(name="6M_video")
chanvid6=chanvid6.fillna(value=0)

In [14]:
#3 month outcomes together
df3M_sum=df3M.groupby(["channelId"]).sum().reset_index()
df3M_sum=df3M_sum.rename(columns={"viewCount":"3M_viewCount", "likeCount":"3M_likeCount", "commentCount":"3M_commentCount", "dislikeCount":"3M_dislikeCount"})
df3M_sum=df3M_sum.fillna(value=0)
#6 month outcomes together
df6M_sum=df6M.groupby(["channelId"]).sum().reset_index()
df6M_sum=df6M_sum.rename(columns={"viewCount":"6M_viewCount", "likeCount":"6M_likeCount", "commentCount":"6M_commentCount", "dislikeCount":"6M_dislikeCount"})
df6M_sum=df6M_sum.fillna(value=0)

In [15]:
#merge to df_channel
df_channel=pd.merge(df_channel, chanvid, how="outer")
df_channel=pd.merge(df_channel, df3M_sum, how="outer")
df_channel=pd.merge(df_channel, chanvid6, how="outer")
df_channel=pd.merge(df_channel, df6M_sum, how="outer")

In [16]:
df_channel.columns

Index(['channelId', 'title', 'collectedTime', 'category', 'sub_category',
       'description', 'publishedAt', 'thumbnailUrl', 'country', 'viewCount',
       'commentCount', 'subscriberCount', 'hiddenSubscriberCount',
       'videoCount', 'keywords', 'unsubscribedTrailer', 'likeCount',
       'dislikeCount', '3M_video', '3M_viewCount', '3M_likeCount',
       '3M_dislikeCount', '3M_commentCount', '6M_video', '6M_viewCount',
       '6M_likeCount', '6M_dislikeCount', '6M_commentCount'],
      dtype='object')

In [17]:
#create final dataset
df_channel_k=df_channel
df_channel_k["산출물수"]=df_channel["videoCount"]
df_channel_k["최근산출물수"]=df_channel["3M_video"]
df_channel_k["정량개선도"]=(df_channel["3M_video"])/(df_channel["6M_video"])
df_channel_k["구독자수"]=df_channel.subscriberCount
df_channel_k["총조회수"]=df_channel.viewCount
df_channel_k["총좋아요수"]=df_channel.likeCount
df_channel_k["총싫어요수"]=df_channel.dislikeCount
df_channel_k["총댓글수"]=df_channel.commentCount
df_channel_k["MAU"]=(df_channel.viewCount)/(df_channel.subscriberCount)
df_channel_k["최근조회수"]=df_channel["3M_viewCount"]
df_channel_k["최근좋아요수"]=df_channel["3M_likeCount"]
df_channel_k["최근싫어요수"]=df_channel["3M_dislikeCount"]
df_channel_k["최근댓글수"]=df_channel["3M_commentCount"]
df_channel_k["조회수개선도"]=df_channel_k["최근조회수"]/df_channel_k["6M_viewCount"]
df_channel_k["좋아요개선도"]=df_channel_k["최근좋아요수"]/df_channel_k["6M_likeCount"]
df_channel_k["댓글수개선도"]=df_channel_k["최근댓글수"]/df_channel_k["6M_commentCount"]
df_channel_k["정성개선도"]=df_channel_k["조회수개선도"]+df_channel_k["좋아요개선도"]+df_channel_k["댓글수개선도"]
df_channel_k=df_channel_k.reset_index()
df_channel_k=df_channel_k.drop(['index','thumbnailUrl', 'country', 'viewCount', 'commentCount', 'subscriberCount', 'hiddenSubscriberCount', 'videoCount', 'keywords', 'unsubscribedTrailer', 'likeCount', 'dislikeCount', '3M_video', "3M_viewCount", "3M_likeCount", "3M_dislikeCount","3M_commentCount"], axis=1)
df_channel_k=df_channel_k.rename(columns={"channelId":"채널ID", "title":"채널명", "collectedTime":"수집일자", "category":"분류1", "sub_category":"분류2", "description":"채널설명", "publishedAt":"채널생성일","hiddenSubscriberCount":"구독자숨김"})
df_channel_k.columns

Index(['채널ID', '채널명', '수집일자', '분류1', '분류2', '채널설명', '채널생성일', '6M_video',
       '6M_viewCount', '6M_likeCount', '6M_dislikeCount', '6M_commentCount',
       '산출물수', '최근산출물수', '정량개선도', '구독자수', '총조회수', '총좋아요수', '총싫어요수', '총댓글수',
       'MAU', '최근조회수', '최근좋아요수', '최근싫어요수', '최근댓글수', '조회수개선도', '좋아요개선도',
       '댓글수개선도', '정성개선도'],
      dtype='object')

In [18]:
#create out of 100 scores
cols=['산출물수', '최근산출물수', '정량개선도', '구독자수', '총조회수', '총좋아요수', '총싫어요수', '총댓글수', 'MAU', '최근조회수', '최근좋아요수', '최근싫어요수', '최근댓글수']#columns for 100 generation
for item in cols:
    item_100=item+"_100"
    df_channel_k[item_100]=(df_channel_k[item]-df_channel_k[item].min())/((df_channel_k[item].max()-df_channel_k[item].min())/100)#최소값을 0점, 최대값을 100점이라 하였을때 상대적인 점수
    

In [19]:
#create standardized score using all channel
zcols=['산출물수', '최근산출물수', '정량개선도', '구독자수', '총조회수', '총좋아요수', '총싫어요수', '총댓글수', 'MAU', '최근조회수', '최근좋아요수', '최근싫어요수', '최근댓글수']#columns for z-score generation
for item in zcols:
    item_Tscore=item+"_zscore"
    df_channel_k[item_Tscore]=((df_channel_k[item] - df_channel_k[item].mean())/(df_channel_k[item].std()))#z-score

In [20]:
# #calculate scores using T-scores
# df_channel_k["MAU_Tscore"]=(df_channel_k["총조회수_Tscore"])/(df_channel_k["구독자수_Tscore"])
# df_channel_k["정량개선도_Tscore"]=(df_channel_k["정기성_Tscore"])/(df_channel_k["산출물수_Tscore"])
# df_channel_k["조회수개선도_Tscore"]=df_channel_k["최근조회수_Tscore"]/df_channel_k["총조회수_Tscore"]
# df_channel_k["좋아요개선도_Tscore"]=df_channel_k["최근좋아요수_Tscore"]/df_channel_k["총좋아요수_Tscore"]
# df_channel_k["댓글수개선도_Tscore"]=df_channel_k["최근댓글수_Tscore"]/df_channel_k["총댓글수_Tscore"]

In [21]:
#최종계산_100
df_channel_k["정량지수_100"]=(df_channel_k['산출물수_100']+df_channel_k['최근산출물수_100']+df_channel_k['정량개선도_100'])/3
df_channel_k["정성지수_100"]=(df_channel_k['구독자수_100']+df_channel_k['총조회수_100']+df_channel_k['총좋아요수_100']+df_channel_k['MAU_100'])/4
df_channel_k["종합홍보지수_100"]=df_channel_k["정량지수_100"]*0.29+df_channel_k["정성지수_100"]*0.71

In [22]:
#최종계산_zscore
df_channel_k["정량지수_zscore"]=(df_channel_k['산출물수_zscore']+df_channel_k['최근산출물수_zscore']+df_channel_k['정량개선도_zscore'])/3
df_channel_k["정성지수_zscore"]=(df_channel_k['구독자수_zscore']+df_channel_k['총조회수_zscore']+df_channel_k['총좋아요수_zscore']+df_channel_k['MAU_zscore'])/4
df_channel_k["종합홍보지수_zscore"]=df_channel_k["정량지수_zscore"]*0.29+df_channel_k["정성지수_zscore"]*0.71

In [23]:
#drop columns not open for clients
df_channel_k=df_channel_k.drop(['6M_viewCount', '6M_video','6M_likeCount', '6M_dislikeCount', '6M_commentCount','최근산출물수', '총댓글수', '조회수개선도', '좋아요개선도', '댓글수개선도','정성개선도', '산출물수_100', '최근산출물수_100', '구독자수_100', '총조회수_100',
       '총좋아요수_100', '총싫어요수_100', '총댓글수_100', 'MAU_100', '최근조회수_100',
       '최근좋아요수_100', '최근싫어요수_100', '최근댓글수_100', '산출물수_zscore', '최근산출물수_zscore',
       '구독자수_zscore', '총조회수_zscore', '총좋아요수_zscore',
       '총싫어요수_zscore', '총댓글수_zscore', 'MAU_zscore', '최근조회수_zscore',
       '최근좋아요수_zscore', '최근싫어요수_zscore', '최근댓글수_zscore'], axis=1)

In [24]:
df_channel_k.to_csv("JHJ.csv")